In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preliminary Preparation

___

## Dependent Libraries Import

In [22]:
# data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# text-processing libraries
import re
import string
import nltk
from nltk.corpus import stopwords

## Accelerator Detection

In [23]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Data Preparation

---

## Data Extraction

In [24]:
train = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")
print('train set shape: ', train.shape)

test = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
print('Test set shape: ', test.shape)

sample_submission = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv")
print('Sample submission set shape: ', sample_submission.shape)

In [25]:
train.head()

In [26]:
train.info()

In [27]:
test.head()

In [28]:
test.info()

In [29]:
sample_submission.head()

# Regular Text Processing

---

## Text Clean 

In [30]:
'''
    Make text lowercase, remove text in square brackets,
    remove links, remove punctuation and remove words containing numbers.
    
'''
def text_cleaner(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

## Text Processing

In [31]:
"""
    Cleaning and parsing the text.
    
"""
def text_preprocessor(text):
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = text_cleaner(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    # remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

In [32]:
train['text_clean'] = train['context'].apply(str).apply(lambda x: text_preprocessor(x))

train['context'].head()

In [33]:
test['text_clean'] = test['context'].apply(str).apply(lambda x: text_preprocessor(x))
test['context']

# Data Visualization

---

## Pie Chart

In [34]:
train_groupby = train.groupby(by = ['language']).count()
train_groupby

In [35]:
sns.color_palette('pastel')

In [36]:
# define Seaborn color palette to use
colors = sns.color_palette('pastel')[2 : 3] + sns.color_palette('pastel')[-1 : ]

# create text props
textprops = dict(
    horizontalalignment = "center",
    verticalalignment = "top",
    rotation = 0,
    rotation_mode = "anchor",
    # rotation_mode = "default",
    size = 14,
    color = sns.color_palette('pastel')[-5],
    # color = "#81D8D0"
)
# create pie chart
plt.figure(figsize=(6, 6))
plt.pie(
    x = train_groupby.id,
    labels = train_groupby.index,
    colors = colors,
    autopct='%.2f%%',
    explode = [0.02] * 2,
    # explode = (0.02, 0.02),
    startangle = 90,
    pctdistance = 0.4,
    labeldistance = 1.2,
    textprops = textprops,
       )
legend = plt.legend(
    title = "Distinguish Samples by Language - Pie Chart",
    title_fontsize = 'x-large',
    fontsize = 'large',
    loc = "lower center",
    bbox_to_anchor = (0.25, -0.2, 0.5, 0.5),
    labelcolor = sns.color_palette('pastel')[-5],
    facecolor = '#F6F8ED',
    edgecolor = sns.color_palette('pastel')[1],
          )
plt.setp(legend.get_title(), color = sns.color_palette('pastel')[3])

# draw circle
centre_circle = plt.Circle((0,0),0.70,fc = 'white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.show()